Concatenate JSON files

In [ ]:
#edit to change data directory
data_directory = "/Users/carljohanson/Desktop/Speciale - Code Project/Code/data/InfoDesk"

name = "daily_MEMO_data" #adjust to change name of CSV file
directory = os.fsencode(data_directory)

include_fields = {
    "description": "strings"
}

out = []

header = ["profile", "date"].extend(include_fields.keys())

for file in os.listdir(directory):

    filename = os.fsdecode(file)

    if filename.endswith(".json"):

        path= os.path.join(data_directory, filename)

        print(f"Reading file: {path}")

        with open(path, "r") as f:
            data = json.loads(f.read())

        results = data["results"]
        profile = data["profile"]

        for match in results:

            data = []

            raw_date = match["date"]

            data.append(str(pd.to_datetime(raw_date)))
            data.append(profile)

            inverted_fields = {}

            try: fields = match.get("fields")
            except KeyError: continue

            if not fields: continue
            for field in fields:

                field_id = field["id"]

                if field_id in include_fields:

                    inverted_fields[field_id] = field[include_fields[field_id]][0] #method for extraction should be a lambda in the include fields value


            data.extend(map(lambda idx: inverted_fields.get(idx, ""), include_fields))

            out.append(data)

        #delete the files after they have been processed
        os.remove(path)

file_path = os.path.join(data_directory, f"{name}")

with open(file_path, "w") as f:
    for m in out:
        try:
            f.write("\t".join(m)+"\n")

        except UnicodeEncodeError:
            print("Unicode error:", m)
            continue